In [420]:
from math import *
import numpy as np
from collections import Counter

In [421]:
"""PARAMETERS

- General:

        n: code length
        k: code dimension
        q: finite field size

- Instance:

        c: codeword

        w: weight

        m_set: {v_0 : m_0 , ... , v_q-1 : m_q-1} 

                v_i : finite field value
                m_i : how many times v_i repeats in c

- Attack:

        n_sol : number of solution of 
        
        L1,L2 : input lists size
        
        rf : it's equal to the square root of n_sol and indicates that only a rf-fraction of the input lists is considered

        Lout : output list size



"""


q = 127
k = q-1
n = k*2

In [422]:
""" RANDOM VECTOR """

def generate_random_vector(q, n):
    return np.random.randint(0, q, size=n)

def psi(c):
    return list(Counter(c).values())


def psi_2(c,q):
    return list(Counter(c).values())

In [423]:
""""ATTACK

    Instance of PECK
    - u in Fq^k
    - G in Fq^(k x n)

    - Find such that    
    


    ------------------ STEP 1 : Partial Gauss Elimination ------------------
    
        Choose J of size k+ell containing the Information set and bring H into quasy-systematic form 
        (Consequently J' = {0,...,n-1} \ J)
        
                                        k+ell       n-(k+ell)
        H_tilde  =  [ H_J | H_J' ] = [   H_1    ][     0     ] ell 
                                     [   H_2    ][     I     ] n-k-ell

        Thus the instance of the problem can be represented as follows:

            - π(c) H_tilde^T = 0 ∈ Fq^n-k --> [c_J | c_J'] H_tilde^T = 0 

            - c_J H_1^T = 0         (1)
              c_J H_2^T + c_J' = 0  (2)
        
        This step costs: 
                                            
    ------------------ STEP 2 : Find codeword with a specific distribution ------------------

        This step costs: 


    ------------------ STEP 3 : Enumerate input lists for collision search ------------------
    

        Bring coordinate J into J_1 J_2 obtaining the following decomposition:

            π(c) = [ c_J1 | c_J2 | c_J' ]

            H_tilde = [ H_J1 | H_J2 ][     0     ]
                      [     H_2     ][     I     ]

        Then create the 2 size lists by arranging all possible values of original vector c in |J_1| and |J_2| entries, respectively.
    
        With l_1 = |J_1| and l_2 = |J_2|

        It is necessary to create only a fraction of the two lists, equal to the square root of the number of solutions. 
        This approach allows you to reduce the cost of enumeration and still detect at least one solution, on averageù

        |L1| = A(l_1, mset) / √N_sol 
        |L2| = A(l_2, mset) / √N_sol 

        This step costs: |L1| + |L2|

    ------------------ STEP 4: Merge Lists ------------------
        
        The two lists are firts sorted according syndrome value on ell symbols.
        Then each pair (c_J1 , c_J2) in L1 x L2 such that c_J1 H_J1^T  = - c_J2 H_J2^T , i.e. a solution for the equation (1) and then check
        if completes to a solution for equation (2)

        Considering that for each element of the first list we need to check on average |L2|/q^ell element of the second list, this step costs:
        |L1||L2| / q^ell


"""

<>:14: SyntaxWarning: invalid escape sequence '\ '
<>:14: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_3390/1287427354.py:14: SyntaxWarning: invalid escape sequence '\ '
  (Consequently J' = {0,...,n-1} \ J)


'"ATTACK\n\n    Instance of PECK\n    - u in Fq^k\n    - G in Fq^(k x n)\n\n    - Find such that    \n\n\n\n    ------------------ STEP 1 : Partial Gauss Elimination ------------------\n\n        Choose J of size k+ell containing the Information set and bring H into quasy-systematic form \n        (Consequently J\' = {0,...,n-1} \\ J)\n\n                                        k+ell       n-(k+ell)\n        H_tilde  =  [ H_J | H_J\' ] = [   H_1    ][     0     ] ell \n                                     [   H_2    ][     I     ] n-k-ell\n\n        Thus the instance of the problem can be represented as follows:\n\n            - π(c) H_tilde^T = 0 ∈ Fq^n-k --> [c_J | c_J\'] H_tilde^T = 0 \n\n            - c_J H_1^T = 0         (1)\n              c_J H_2^T + c_J\' = 0  (2)\n\n        This step costs: \n\n    ------------------ STEP 2 : Find codeword with a specific distribution ------------------\n\n        This step costs: \n\n\n    ------------------ STEP 3 : Enumerate input lists for 

In [424]:
""" ARRANGEMENTS """

"""
Count iteratively the number of all the possible distinct vectors of size k obtained by arrange k values of vector c 
"""
def arrangements_mset(cnt: list, k: int):

    nrs = [0 for _ in range(k+1)]
    nrs[0]=1

    for n in range(0,len(cnt)):
        for l in range(k,0,-1):
            nr = 0
            bino = 1
            for i in range(min(cnt[n],l)+1):
                nr = nr + bino*nrs[l-i]
                bino = int(bino * (l-i)/(i+1))
            nrs[l] = nr
    #print(f"\n mult:Result {cnt} on {k} values, {nrs[-1]}")
    return log(nrs[-1],2)

"""
Compute multinomial coefficient 
"""
def compute_multinomial(n, m_set):
    n_ = log(factorial(n), 2)
    d_ = 0
    for mi in m_set:
        d_ = d_ + log(factorial(mi), 2)
    return n_ - d_


In [425]:
""" LISTS SIZE """


"""
Compute the number of solutions of a given instance of PEK (c,H) as:

                Number of Permutation of c
    #NumSol = ---------------------------------- =
                        Code Size


"""

def compute_n_sol(n, k, q, m_set):
    dim_ = q**(n-k)
    exp_mult = compute_multinomial(n, m_set)
    exp_dim = log(dim_,2)
    
    n_sol = 1 +( 2**(exp_mult-exp_dim) - 1/(q**(n-k))) 
    return n_sol


"""

The lists contains a rf-fraction of all the possible distinct vectors of size k obtained by arrange k values of vector c 
where rf it's the square root of the number of solution

"""
def list_creation_cost(m_set, l):
    list_size = arrangements_mset(m_set, l)
    return list_size


In [426]:
"""ATTACK"""

def attack_cost(n, k, q, max_ell, m_set):

    n_sol =  compute_n_sol(n, k, q, m_set)
    b_res = ((0, 0, 0), 0, 1000)
    rf = log(sqrt(n_sol),2)
    n_sol = log(n_sol,2)
    for ell in range(0, max_ell+1):
        #print(f"\n ell {ell}")
        try:
            
            l1 = int((k+ell)/2)
            l2 = k+ell-l1
            L1 = list_creation_cost(m_set, l1)
            L2 = list_creation_cost(m_set, l2)
            L1_red = L1-rf
            L2_red = L2-rf
            p_coll = log(q**ell,2)
            L_out = L1_red+L2_red-p_coll
            c = max([L1_red, L2_red, L_out])
            #print(ell, L1, L2, L_out)
            #print(f"L1: {L1}, L2: {L2}, p_coll: {p_coll}, L_out: {L_out}")
        # print(ell, L1,L2,p_coll)
            _, _, bc = b_res
            if c < bc:

                b_res = ((L1, L2, L_out), ell, c)
        except Exception as e:
            print(e)
            #print("FAIL")
            pass
    return b_res, n_sol


In [427]:
""" Probability to find a subset of Fq^n given a ceratin multiplicity """
def num_strings_with_exactly_d_symbols(n, q, d):
    if d > q or d > n:
        return 0
    total = 0
    for k in range(d + 1):
        total += (-1)**k * comb(d, k) * (d - k)**n
    return comb(q, d) * total

def num_strings_with_at_most_d_symbols(n, q, d):
    total = 0
    for i in range(1, d + 1):
        total += num_strings_with_exactly_d_symbols(n, q, i)
    return total

def num_strings_with_less_than_d_symbols(n, q, d):
    return num_strings_with_at_most_d_symbols(n, q, d - 1)

def num_strings_with_more_than_d_symbols(n, q, d):
    total_all = q ** n
    total_at_most_d = num_strings_with_at_most_d_symbols(n, q, d)
    return total_all - total_at_most_d




In [428]:
"""Find d for which T_Sample is > 2**128"""
def find_limit_d(q):
    for d in range(2,q+1):
        dim_set = log(num_strings_with_less_than_d_symbols(n, q, d),2)
        prob = (dim_set-log(q**n, 2))
        if(prob>-128):
            return d, prob
    return q, -128

In [429]:
"""LOWER BOUND FOR THE ATTACK"""

def get_m_star(t):
    m_star = list()
    m_star = [1 for _ in range(t)]
    m_star.append(0) 
    return m_star

def get_ell_tilde(t, m_star):
    return int((t)/2 - log(2**arrangements_mset(m_star, t),q)/2) 

def get_tilde_m_star(best_t):
    tilde_m_star = list()
    tilde_m_star = [1 for _ in range(best_t)]+[0 for _ in range(q-best_t)]
    return tilde_m_star

def lower_bound(n, k, tilde_m_star, best_t):
    denom = arrangements_mset(tilde_m_star,best_t)/2
    num = log(q**(n-k),2)/2
    return num-denom


1

1

In [430]:
c = generate_random_vector(q, n)
m_set = psi(c)
d = len(m_set)
max_ell = n-k
((L1, L2, L_out), ell, c), n_sol = attack_cost(n, k, q, max_ell, m_set)

print(f"--- SOLVER TIME COMPLEXITY ---\nVector with {d} distinct values\nNumber of Solutions--> {n_sol} \nLists Size--> L1: 2^{int(L1)}, L2:2^{int(L2)}, L_out:2^{int(L_out)}\nFor ell: {ell} Binary Cost --> 2^{int(c)} ")


--- SOLVER TIME COMPLEXITY ---
Vector with 111 distinct values
Number of Solutions--> 580.2958413958684 
Lists Size--> L1: 2^530, L2:2^530, L_out:2^242
For ell: 34 Binary Cost --> 2^242 


In [431]:
best_d, prob = find_limit_d(q)
iterations = -prob

print(f"When d<{best_d}, T_Sample<2^{int(iterations)}")

When d<65, T_Sample<2^127


In [432]:
t = n-k
m_star = get_m_star(t)
ell_tilde = get_ell_tilde(t, m_star)
best_t = n-k-ell_tilde
tilde_m_star =get_tilde_m_star(best_t)
lb_c = lower_bound(n, k, tilde_m_star, best_t)


print(f"Best ell: {ell_tilde}\nLower Bound for the Time Complexity: 2^{int(lb_c)}")

Best ell: 12
Lower Bound for the Time Complexity: 2^130
